In [39]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from dateutil.parser import parse
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from tqdm import tqdm

In [40]:
transactions_full = pd.read_csv('./data/transactions_test.csv', parse_dates=["TX_TS"])
# transactions_train_og = transactions_train_og.set_index("TX_ID")

# merchants_full = pd.read_csv('./data/merchants.csv')

# terminals_full = pd.read_csv('./data/terminals.csv')

# customers_full = pd.read_csv('./data/customers.csv')

In [41]:
transactions_full = pd.merge(transactions_full,terminals_full,on='TERMINAL_ID')

In [42]:
transactions_full = pd.merge(transactions_full,customers_full,on='CUSTOMER_ID')

In [43]:
transactions_full["TX_TS"] = pd.to_datetime(transactions_full["TX_TS"],format='%Y-%m-%d')

In [44]:
def is_weekend(tx_datetime):
    
    # Transform date into weekday (0 is Monday, 6 is Sunday)
    weekday = tx_datetime.weekday()
    # Binary value: 0 if weekday, 1 if weekend
    is_weekend = weekday>=5
    
    return int(is_weekend)
transactions_full['TX_DURING_WEEKEND']=transactions_full.TX_TS.apply(is_weekend)


In [45]:
def is_night(tx_datetime):
    
    # Get the hour of the transaction
    tx_hour = tx_datetime.hour
    # Binary value: 1 if hour less than 6, and 0 otherwise
    is_night = tx_hour<=6
    
    return int(is_night)
transactions_full['TX_DURING_NIGHT']=transactions_full.TX_TS.apply(is_night)


In [46]:
BEGIN_DATE = pd.to_datetime("2021-01-01 00:00:08+00:00") 
END_DATE = pd.to_datetime("2022-01-01 00:13:26+00:00")

In [47]:
transactions_full['TX_TIME_SECONDS'] = (transactions_full['TX_TS'] - BEGIN_DATE).astype('timedelta64[s]')
transactions_full['TX_TIME_DAYS'] = (transactions_full['TX_TS'] - BEGIN_DATE).astype('timedelta64[h]')/24

In [48]:
def get_customer_spending_behaviour_features(customer_transactions, windows_size_in_days=[1,7,30]):
    
    # Let us first order transactions chronologically
    customer_transactions=customer_transactions.sort_values('TX_TS')
    
    # The transaction date and time is set as the index, which will allow the use of the rolling function 
    customer_transactions.index=customer_transactions.TX_TS
    
    # For each window size
    for window_size in windows_size_in_days:
        
        # Compute the sum of the transaction amounts and the number of transactions for the given window size
        SUM_AMOUNT_TX_WINDOW=customer_transactions['TX_AMOUNT'].rolling(str(window_size)+'d').sum()
        NB_TX_WINDOW=customer_transactions['TX_AMOUNT'].rolling(str(window_size)+'d').count()
    
        # Compute the average transaction amount for the given window size
        # NB_TX_WINDOW is always >0 since current transaction is always included
        AVG_AMOUNT_TX_WINDOW=SUM_AMOUNT_TX_WINDOW/NB_TX_WINDOW
    
        # Save feature values
        customer_transactions['CUSTOMER_ID_NB_TX_'+str(window_size)+'DAY_WINDOW']=list(NB_TX_WINDOW)
        customer_transactions['CUSTOMER_ID_AVG_AMOUNT_'+str(window_size)+'DAY_WINDOW']=list(AVG_AMOUNT_TX_WINDOW)
    
    # Reindex according to transaction IDs
    customer_transactions.index=customer_transactions.TX_ID
        
    # And return the dataframe with the new features
    return customer_transactions

In [49]:
transactions_full=transactions_full.groupby('CUSTOMER_ID').apply(lambda x: get_customer_spending_behaviour_features(x, windows_size_in_days=[1,7,30]))

In [50]:
def get_count_risk_rolling_window(terminal_transactions, delay_period=7, windows_size_in_days=[1,7,30], feature="TERMINAL_ID"):
    
    terminal_transactions=terminal_transactions.sort_values('TX_TS')
    
    terminal_transactions.index=terminal_transactions.TX_TS
    
    NB_FRAUD_DELAY=terminal_transactions['TX_FRAUD'].rolling(str(delay_period)+'d').sum()
    NB_TX_DELAY=terminal_transactions['TX_FRAUD'].rolling(str(delay_period)+'d').count()
    
    for window_size in windows_size_in_days:
    
        NB_FRAUD_DELAY_WINDOW=terminal_transactions['TX_FRAUD'].rolling(str(delay_period+window_size)+'d').sum()
        NB_TX_DELAY_WINDOW=terminal_transactions['TX_FRAUD'].rolling(str(delay_period+window_size)+'d').count()
    
        NB_FRAUD_WINDOW=NB_FRAUD_DELAY_WINDOW-NB_FRAUD_DELAY
        NB_TX_WINDOW=NB_TX_DELAY_WINDOW-NB_TX_DELAY
    
        RISK_WINDOW=NB_FRAUD_WINDOW/NB_TX_WINDOW
        
        terminal_transactions[feature+'_NB_TX_'+str(window_size)+'DAY_WINDOW']=list(NB_TX_WINDOW)
        terminal_transactions[feature+'_RISK_'+str(window_size)+'DAY_WINDOW']=list(RISK_WINDOW)
        
    terminal_transactions.index=terminal_transactions.TX_ID
    
    # Replace NA values with 0 (all undefined risk scores where NB_TX_WINDOW is 0) 
    terminal_transactions.fillna(0,inplace=True)
    
    return terminal_transactions

In [51]:
transactions_df=transactions_full.groupby('TERMINAL_ID').apply(lambda x: get_count_risk_rolling_window(x, delay_period=7, windows_size_in_days=[1,7,30], feature="TERMINAL_ID"))
transactions_df=transactions_df.sort_values('TX_TS').reset_index(drop=True)

KeyError: 'TX_FRAUD'

In [ ]:
transactions_full = transactions_df

In [ ]:
transactions_full.head()

,TX_ID,TX_TS,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,TRANSACTION_GOODS_AND_SERVICES_AMOUNT,TRANSACTION_CASHBACK_AMOUNT,CARD_EXPIRY_DATE,CARD_DATA,...,CUSTOMER_ID_NB_TX_7DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,CUSTOMER_ID_NB_TX_30DAY_WINDOW,CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW,TERMINAL_ID_NB_TX_1DAY_WINDOW,TERMINAL_ID_RISK_1DAY_WINDOW,TERMINAL_ID_NB_TX_7DAY_WINDOW,TERMINAL_ID_RISK_7DAY_WINDOW,TERMINAL_ID_NB_TX_30DAY_WINDOW,TERMINAL_ID_RISK_30DAY_WINDOW
0,a0d7c943b544215745e13ebce08a4b8276168767,2021-01-01 00:00:08+00:00,2108277906068006,27974316,78.98,0,78.98,0.0,04/21,4485********975,...,1.0,78.98,1.0,78.98,0.0,0.0,0.0,0.0,0.0,0.0
1,c06aa650651e1402c16ddf309d61516de22d0331,2021-01-01 00:00:14+00:00,8662651220152351,46261637,46.42,0,46.42,0.0,04/21,4485********444,...,1.0,46.42,1.0,46.42,0.0,0.0,0.0,0.0,0.0,0.0
2,09a966ffc5878eb652d3f36496cdb342d7576dd6,2021-01-01 00:00:15+00:00,9508987894574818,21664478,38.84,0,38.84,0.0,01/23,4916********672,...,1.0,38.84,1.0,38.84,0.0,0.0,0.0,0.0,0.0,0.0
3,5689648bed268990eb90c11fb9a7902f2d1b2cf2,2021-01-01 00:01:08+00:00,3040941832365513,60286185,57.74,0,57.74,0.0,07/22,4024********300,...,1.0,57.74,1.0,57.74,0.0,0.0,0.0,0.0,0.0,0.0
4,541d5014993dafacfc25e54defd9c42a1cb580ce,2021-01-01 00:02:00+00:00,3014216432000074,56911243,99.13,0,99.13,0.0,03/22,4929********121,...,1.0,99.13,1.0,99.13,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
# Divide data into training and validation subsets
y = transactions_full['TX_FRAUD']
X = transactions_full.drop(['TX_FRAUD'],axis = 1)
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.7, test_size=0.3,random_state=42)

In [38]:
location = 'feature_set_3'


transactions_full.to_csv(F'./data/{location}/transactions_full.csv',index=False)
X_train_full.to_csv(F'./data/{location}/X_train_full.csv',index=False)
X_valid_full.to_csv(F'./data/{location}/X_valid_full.csv',index=False)
y_train.to_csv(F'./data/{location}/y_train.csv',index=False)
y_valid.to_csv(F'./data/{location}/y_valid.csv',index=False)